[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 -16,
 -15,
 -14,
 -13,
 -12,
 -11,
 -10,
 -9,
 -8,
 -7,
 -6,
 -5,
 -4,
 -3,
 -2,
 -1]

In [3]:
import torch

@torch.jit.script
def beam_search(log_probs: torch.Tensor, beam_size: int):
    """
    Performs beam search on a tensor of log probabilities.

    Args:
        log_probs (torch.Tensor): Tensor of shape (b, t, v) containing log probabilities.
        beam_size (int): Number of beams to keep at each time step.

    Returns:
        sequences (torch.Tensor): Tensor of shape (b, beam_size, t) containing the top sequences.
        scores (torch.Tensor): Tensor of shape (b, beam_size) containing the scores of the top sequences.
    """
    
    b, t, v = log_probs.size()
    
    initial_beam_size = min(beam_size, v) # At the very first step (time step 0), we can't have more beams than the vocabulary size. This line ensures that the initial number of beams considered doesn't exceed the number of possible first tokens.

    topk_scores, topk_indices = torch.topk(log_probs[:, 0, :], initial_beam_size, dim=-1) # Returns the k largest elements of the given input tensor along a given dimension
    sequences = topk_indices.unsqueeze(-1)  # (b, initial_beam_size, 1)
    scores = topk_scores  # (b, initial_beam_size)

    for step in range(1, t):
        # Expand the current sequences with all possible next tokens
        current_log_probs = log_probs[:, step, :].unsqueeze(1)  # (b, 1, v)
        expanded_scores = scores.unsqueeze(-1) + current_log_probs  # (b, beam_size, v)
        flat_scores = expanded_scores.view(b, -1)  # (b, beam_size * v)

        # Select the top-k scores and their corresponding indices
        topk_flat_scores, topk_indices = flat_scores.topk(beam_size, dim=-1)  # (b, beam_size)
        beam_indices = topk_indices // v  # Indices of sequences to expand
        token_indices = topk_indices % v  # New tokens to append

        # Gather the sequences to expand and append the new tokens
        sequences = torch.gather(sequences, 1, beam_indices.unsqueeze(-1).expand(-1, -1, sequences.size(-1)))
        sequences = torch.cat([sequences, token_indices.unsqueeze(-1)], dim=-1)  # (b, beam_size, step+1)

        # Update the scores
        scores = topk_flat_scores

    return sequences, scores.unsqueeze(-1)


In [4]:
batch_size = 128
sequence_length = 100
vocab_size = 29
beam_size = 2

# Simulate log probabilities
log_probs = torch.randn(batch_size, sequence_length, vocab_size).log_softmax(dim=-1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
log_probs = log_probs.to(device)

# Perform beam search
sequences, scores = beam_search(log_probs, beam_size)

print("Top sequences:", sequences) # bsz, beamsize,seq_len
print("Scores:", scores) # bsz, beamsize,1

Top sequences: tensor([[[18, 26, 25,  ..., 12, 24, 26],
         [18, 26, 25,  ..., 12, 24, 26]],

        [[ 9, 27,  7,  ...,  3, 26,  3],
         [ 9, 27,  7,  ...,  3, 26,  3]],

        [[24,  4, 17,  ..., 26, 17,  5],
         [24,  4, 17,  ..., 26, 17,  5]],

        ...,

        [[17, 24, 26,  ..., 18,  3, 25],
         [17, 24, 26,  ..., 18,  3, 25]],

        [[19,  3, 15,  ..., 20, 23, 13],
         [19,  3, 15,  ..., 20, 23, 13]],

        [[15,  2, 22,  ..., 15, 25, 19],
         [15,  2, 22,  ..., 15, 25, 19]]], device='cuda:0')
Scores: tensor([[[-186.6484],
         [-186.6507]],

        [[-181.7525],
         [-181.7560]],

        [[-175.4822],
         [-175.4828]],

        [[-179.9598],
         [-179.9617]],

        [[-180.6495],
         [-180.6533]],

        [[-179.3754],
         [-179.3794]],

        [[-181.5901],
         [-181.6168]],

        [[-179.5302],
         [-179.5359]],

        [[-180.6545],
         [-180.6581]],

        [[-181.6397],
      

In [5]:
sequences.shape, scores.shape

(torch.Size([128, 2, 100]), torch.Size([128, 2, 1]))

In [6]:
path_probs = torch.gather(log_probs, 2, sequences.transpose(1,2)).transpose(1,2) # bsz, beamsize, T
path_probs

tensor([[[-1.9489, -1.2621, -1.9904,  ..., -2.1885, -1.6692, -2.1089],
         [-1.9489, -1.2621, -1.9904,  ..., -2.1885, -1.6692, -2.1089]],

        [[-2.1763, -1.1933, -1.4817,  ..., -1.6028, -2.1425, -1.3587],
         [-2.1763, -1.1933, -1.4817,  ..., -1.6028, -2.1425, -1.3587]],

        [[-1.9329, -1.3201, -1.9363,  ..., -1.7841, -1.7984, -1.6497],
         [-1.9329, -1.3201, -1.9363,  ..., -1.7841, -1.7984, -1.6497]],

        ...,

        [[-2.4142, -1.6109, -1.7212,  ..., -1.5578, -1.9366, -1.5802],
         [-2.4142, -1.6109, -1.7212,  ..., -1.5578, -1.9366, -1.5802]],

        [[-2.1040, -1.6718, -2.2350,  ..., -1.6370, -2.1403, -1.6984],
         [-2.1040, -1.6718, -2.2350,  ..., -1.6370, -2.1403, -1.6984]],

        [[-1.5115, -1.7583, -1.9641,  ..., -1.6032, -2.3413, -2.2177],
         [-1.5115, -1.7583, -1.9641,  ..., -1.6032, -2.3413, -2.2177]]],
       device='cuda:0')

In [7]:
mean = scores.mean(dim=1, keepdim=True)
std = scores.std(dim=1, keepdim=True)

scores = (scores - mean) / std

In [8]:
path_probs*scores

tensor([[[-1.3781, -0.8924, -1.4074,  ..., -1.5475, -1.1803, -1.4912],
         [ 1.3781,  0.8924,  1.4074,  ...,  1.5475,  1.1803,  1.4912]],

        [[-1.5322, -0.8401, -1.0432,  ..., -1.1285, -1.5085, -0.9566],
         [ 1.5455,  0.8474,  1.0522,  ...,  1.1382,  1.5215,  0.9649]],

        [[-1.3668, -0.9335, -1.3692,  ..., -1.2615, -1.2717, -1.1665],
         [ 1.3668,  0.9335,  1.3692,  ...,  1.2615,  1.2717,  1.1665]],

        ...,

        [[-1.7334, -1.1566, -1.2358,  ..., -1.1185, -1.3904, -1.1346],
         [ 1.6808,  1.1215,  1.1983,  ...,  1.0846,  1.3483,  1.1002]],

        [[-1.4877, -1.1822, -1.5804,  ..., -1.1575, -1.5134, -1.2009],
         [ 1.4877,  1.1822,  1.5804,  ...,  1.1575,  1.5134,  1.2009]],

        [[-1.0688, -1.2433, -1.3888,  ..., -1.1337, -1.6556, -1.5681],
         [ 1.0688,  1.2433,  1.3888,  ...,  1.1337,  1.6556,  1.5681]]],
       device='cuda:0')

In [9]:
sequences # bsz,beam,T

tensor([[[18, 26, 25,  ..., 12, 24, 26],
         [18, 26, 25,  ..., 12, 24, 26]],

        [[ 9, 27,  7,  ...,  3, 26,  3],
         [ 9, 27,  7,  ...,  3, 26,  3]],

        [[24,  4, 17,  ..., 26, 17,  5],
         [24,  4, 17,  ..., 26, 17,  5]],

        ...,

        [[17, 24, 26,  ..., 18,  3, 25],
         [17, 24, 26,  ..., 18,  3, 25]],

        [[19,  3, 15,  ..., 20, 23, 13],
         [19,  3, 15,  ..., 20, 23, 13]],

        [[15,  2, 22,  ..., 15, 25, 19],
         [15,  2, 22,  ..., 15, 25, 19]]], device='cuda:0')

In [10]:
# If using PyTorch
import torch

vocab = [' ', "'", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '?']
len(vocab) # 29-1 is the shape 

idx2char = {i:c for i,c in enumerate(vocab)}

def ctc_merge_string(s: str, blank_char='?'):
    merged = []
    prev = None
    for c in s:
        if c == blank_char:
            prev = None  # reset repetition check on blank
            continue
        if c != prev:
            merged.append(c)
        prev = c
    return ''.join(merged)
   

# decode_seq(sequences, vocab)[0]
# sequences.shape torch.Size([128, 2, 100])

In [11]:
# 1) move to CPU & to plain Python list of lists
sentences = []
for b in range(sequences.shape[0]):
    print(b)
    rows = sequences[b].cpu().tolist() # beam,T
    decoded_beams = [ctc_merge_string( ''.join(idx2char[i] for i in row) ) for row in rows]
    sentences.append(decoded_beams)
print(sentences)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
[["QYXULJMRKRBOWXLXEPU DRMIRCJKWPIGTJCHKLBCBY'CSBHNMIGHTPSVOXCRWPXKCAIBDZCOFBCXQRY'BUFWYMXZEBMTMKWY", "QYXULJMRKRBOWXLXEPU DRMIRCJKWPIGTJCHKLBCBY'CSBHNMIGHTPSVOXCRWPXKCAIBDZCOFBCXGRY'BUFWYMXZEBMTMKWY"], ["HZFIZFTWVUZOHNHIJRKOMS'TK DWJL'BZCEMTUHEKPHRTOMIKNSBLSJQRNLFQHDL'LUF JVPNQMSWHEZC' RZYCBWBYB", "HZFIZFTWVUZOHNHIJRKOMS'TK DWJL'BZCEMTUHEKPHRTOMIKNSBLSJQRNLFQHDL'LUF JVPNQM'WHEZC' RZYCBWBYB"], ["WCPXGJKPJBLEAP HKIKMTJKIMJFRDWXPURN'ESURVZG YJYXYCGFLSIHFXZCZUQ'J'CNSZMAWIQWKBILGKVGAZECPTEPFYPD", "WCPXGJKPJBLEAP HKIKMTJKIMJFRDWXPURN'ESURVZG YJYXYCGFLSIHFXZCZUQ'JPCNSZMAWIQWKBILGKVGAZECPTEPFYPD"],

In [12]:
sentences

[["QYXULJMRKRBOWXLXEPU DRMIRCJKWPIGTJCHKLBCBY'CSBHNMIGHTPSVOXCRWPXKCAIBDZCOFBCXQRY'BUFWYMXZEBMTMKWY",
  "QYXULJMRKRBOWXLXEPU DRMIRCJKWPIGTJCHKLBCBY'CSBHNMIGHTPSVOXCRWPXKCAIBDZCOFBCXGRY'BUFWYMXZEBMTMKWY"],
 ["HZFIZFTWVUZOHNHIJRKOMS'TK DWJL'BZCEMTUHEKPHRTOMIKNSBLSJQRNLFQHDL'LUF JVPNQMSWHEZC' RZYCBWBYB",
  "HZFIZFTWVUZOHNHIJRKOMS'TK DWJL'BZCEMTUHEKPHRTOMIKNSBLSJQRNLFQHDL'LUF JVPNQM'WHEZC' RZYCBWBYB"],
 ["WCPXGJKPJBLEAP HKIKMTJKIMJFRDWXPURN'ESURVZG YJYXYCGFLSIHFXZCZUQ'J'CNSZMAWIQWKBILGKVGAZECPTEPFYPD",
  "WCPXGJKPJBLEAP HKIKMTJKIMJFRDWXPURN'ESURVZG YJYXYCGFLSIHFXZCZUQ'JPCNSZMAWIQWKBILGKVGAZECPTEPFYPD"],
 ["FIR'OGHOHEULTJIHW'ICIP'YREQPVS HRUAVUSXBKZBKMNOEAHVNEOTHJEFZUB'WRPS SIYCGHTMWB 'HWY MBADEQXCK",
  "FIR'OGHOHEULTJIHW'ICIP'YREQPVS HRUAVUSXBKZBKMNOEAHVNEOTHJEFZUB'WRPS SIYCGHTMWB 'HWM MBADEQXCK"],
 ["OGMOIBRUBOENIKHVJRMWJDAHLREHIDKYWCFNZCUSUZOKYHAUTRNDAJGCSNYQI'YAOF'CXDGPCHIX'HSAGQYEN QSP'IXZF",
  "OGMOIBRUBOENIKHVJRMWJDAHLREHIDKYWCFNZCUSUZOKYHAUTRNDKJGCSNYQI'YAOF'CXDGPCHIX'HSAGQYEN QSP'I

In [ ]:
"""
1.  beamctc decoder
2. 


"""